<a href="https://colab.research.google.com/github/JesusD03/BillBoard/blob/main/Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Importaciones necesarias
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [53]:
# 1. Cargar el dataset
# Link al dataset: https://www.kaggle.com/datasets/kapturovalexander/bank-credit-scoring?resource=download
file_path = '/content/drive/MyDrive/Colab Notebooks/Financial LLM/datasets/bank.csv'
df = pd.read_csv(file_path, delimiter=';')

In [54]:
# Observar la estructura de los datos
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [56]:
# 2. Convertir variables categóricas a numéricas
# Variables binarias (Sí/No) como 'default', 'housing', 'loan', 'y'
binary_columns = ['default', 'housing', 'loan', 'y']

# Verificar si hay valores nulos antes del mapeo
print("Valores nulos antes del mapeo:")
print(df[binary_columns + ['month']].isnull().sum())

# Verificar valores únicos antes de aplicar map() para asegurar que no haya valores no mapeados
print("Valores únicos antes del mapeo:")
print(df[binary_columns].apply(lambda x: x.unique()))
print(df['month'].unique())

# Aplicar el mapeo
for col in binary_columns:
    df[col] = df[col].map({'yes': 1, 'no': 0})

# Mapear los meses
months = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
          'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
df['month'] = df['month'].map(months)

# Verificar si hay valores nulos después del mapeo
print("Valores nulos después del mapeo:")
print(df[binary_columns + ['month']].isnull().sum())

# Convertir otras variables categóricas con LabelEncoder o OneHotEncoder
# Label Encoding para variables ordinales o categóricas con muchas clases
label_enc = LabelEncoder()
df['job'] = label_enc.fit_transform(df['job'])
df['marital'] = label_enc.fit_transform(df['marital'])
df['education'] = label_enc.fit_transform(df['education'])
df['contact'] = label_enc.fit_transform(df['contact'])
df['poutcome'] = label_enc.fit_transform(df['poutcome'])

Valores nulos antes del mapeo:
default    0
housing    0
loan       0
y          0
month      0
dtype: int64
Valores únicos antes del mapeo:
  default housing loan    y
0      no      no   no   no
1     yes     yes  yes  yes
['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
Valores nulos después del mapeo:
default    0
housing    0
loan       0
y          0
month      0
dtype: int64


In [57]:
# 3. Escalar las características numéricas
scaler = StandardScaler()
num_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
df[num_columns] = scaler.fit_transform(df[num_columns])

In [58]:
# 4. Separar las características (X) y las etiquetas (y)
X = df.drop(columns=['y'])  # Características
y = df['y']  # Etiqueta (la columna objetivo)

In [59]:
# 5. Mostrar la forma de los datos preprocesados
print("Características (X):", X.shape)
print("Etiqueta (y):", y.shape)

Características (X): (4521, 16)
Etiqueta (y): (4521,)


In [60]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [62]:
# Crear el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Para clasificaciones binarias
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7310 - loss: 0.5379 - val_accuracy: 0.8917 - val_loss: 0.2856
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8887 - loss: 0.2864 - val_accuracy: 0.8873 - val_loss: 0.2618
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8851 - loss: 0.2669 - val_accuracy: 0.8895 - val_loss: 0.2515
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8974 - loss: 0.2568 - val_accuracy: 0.8862 - val_loss: 0.2520
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8884 - loss: 0.2567 - val_accuracy: 0.8873 - val_loss: 0.2486
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8936 - loss: 0.2529 - val_accuracy: 0.8928 - val_loss: 0.2420
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8987 - loss: 0.2455 - val_accuracy: 0.8939 - val_loss: 0.2418
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9014 - loss: 0.2393 - val_accuracy: 0.8917 - val_

In [63]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8891 - loss: 0.2403 
Loss: 0.24027058482170105, Accuracy: 0.8972375392913818
